In [1]:
!python -V

Python 3.9.19


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment3")

2024/05/30 22:11:27 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment3' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/mlops/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1717107087030, experiment_id='1', last_update_time=1717107087030, lifecycle_stage='active', name='nyc-taxi-experiment3', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
!ls -l

total 412
-rw-rw-r-- 1 mlops mlops   3734 May 19 22:35 README.md
drwxrwxr-x 2 mlops mlops   4096 May 23 04:59 data
-rw-rw-r-- 1 mlops mlops 139394 May 30 04:48 duration-prediction.ipynb
drwxrwxr-x 2 mlops mlops   4096 May 19 22:35 images
-rw-rw-r-- 1 mlops mlops   1165 May 19 22:35 meta.json
-rw-r--r-- 1 mlops mlops 217088 May 30 22:11 mlflow.db
-rw-rw-r-- 1 mlops mlops   4676 May 19 22:35 mlflow_on_aws.md
drwxrwxr-x 4 mlops mlops   4096 May 28 23:33 mlruns
-rw-rw-r-- 1 mlops mlops  15480 May 19 22:35 model-registry.ipynb
drwxrwxr-x 2 mlops mlops   4096 May 23 05:56 models
-rw-rw-r-- 1 mlops mlops     77 May 19 22:35 requirements.txt
drwxrwxr-x 2 mlops mlops   4096 May 19 22:35 running-mlflow-examples


In [9]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [10]:
len(df_train), len(df_val)

(3009173, 2855951)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.247057977640596

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer", "akash")

    mlflow.log_param("train-data-path", "./data/yellow_tripdata_2023-01.csv")
    mlflow.log_param("valid-data-path", "./data/yellow_tripdata_2023-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:
import xgboost as xgb

In [20]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                    | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:24:09] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.52307                                     
[1]	validation-rmse:5.14990                                     
[2]	validation-rmse:5.10660                                     
[3]	validation-rmse:5.08924                                     
[4]	validation-rmse:5.08169                                     
[5]	validation-rmse:5.07563                                     
[6]	validation-rmse:5.07041                                     
[7]	validation-rmse:5.06490                                     
[8]	validation-rmse:5.05518                                     
[9]	validation-rmse:5.05067                                     
  2%| | 1/50 [00:14<11:41, 14.31s/trial, best loss: 5.0506676127

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:24:23] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.42040                                     
[1]	validation-rmse:8.84809                                     
[2]	validation-rmse:8.34401                                     
[3]	validation-rmse:7.90139                                     
[4]	validation-rmse:7.51429                                     
[5]	validation-rmse:7.17739                                     
[6]	validation-rmse:6.88469                                     
[7]	validation-rmse:6.63151                                     
[8]	validation-rmse:6.41333                                     
[9]	validation-rmse:6.22577                                     
  4%| | 2/50 [00:27<11:04, 13.85s/trial, best loss: 5.0506676127

...

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:00:38] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.20736                                     
[1]	validation-rmse:5.22457                                     
[2]	validation-rmse:4.99091                                     
[3]	validation-rmse:4.91812                                     
[4]	validation-rmse:4.89308                                     
[5]	validation-rmse:4.87892                                     
[6]	validation-rmse:4.87067                                     
[7]	validation-rmse:4.86566                                     
[8]	validation-rmse:4.86085                                     
[9]	validation-rmse:4.85615                                     
 98%|▉| 49/50 [36:56<00:42, 42.11s/trial, best loss: 4.696573500

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:01:06] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.91152                                     
[1]	validation-rmse:7.98792                                     
[2]	validation-rmse:7.25774                                     
[3]	validation-rmse:6.68769                                     
[4]	validation-rmse:6.24625                                     
[5]	validation-rmse:5.90886                                     
[6]	validation-rmse:5.65166                                     
[7]	validation-rmse:5.45801                                     
[8]	validation-rmse:5.31180                                     
[9]	validation-rmse:5.20192                                     
100%|█| 50/50 [37:37<00:00, 45.15s/trial, best loss: 4.696573500


/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [24]:
mlflow.xgboost.autolog(disable=True)

In [25]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.515363222104369,
        'max_depth': 94,
        'min_child_weight': 6.239676007741263,
        'objective': 'reg:linear',
        'reg_alpha': 0.08024205108205405,
        'reg_lambda': 0.002576667534447248,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=5
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:05:16] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:6.54451
[1]	validation-rmse:5.32228
[2]	validation-rmse:4.94071
[3]	validation-rmse:4.81319
[4]	validation-rmse:4.75991
[5]	validation-rmse:4.73472
[6]	validation-rmse:4.71822
[7]	validation-rmse:4.70741
[8]	validation-rmse:4.70018
[9]	validation-rmse:4.69657
[10]	validation-rmse:4.69350
[11]	validation-rmse:4.69098
[12]	validation-rmse:4.68805
[13]	validation-rmse:4.68526
[14]	validation-rmse:4.68264
[15]	validation-rmse:4.67991
[16]	validation-rmse:4.67751
[17]	validation-rmse:4.67542
[18]	validation-rmse:4.67326
[19]	validation-rmse:4.67115
[20]	validation-rmse:4.66909
[21]	validation-rmse:4.66712
[22]	validation-rmse:4.66527
[23]	validation-rmse:4.66321
[24]	validation-rmse:4.66149
[25]	validation-rmse:4.65922
[26]	validation-rmse:4.65733
[27]	validation-rmse:4.65566
[28]	validation-rmse:4.65382
[29]	validation-rmse:4.65246
[30]	validation-rmse:4.65068
[31]	validation-rmse:4.64809
[32]	validation-rmse:4.64662
[33]	validation-rmse:4.64526
[34]	validation-rmse:4.6

/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:10:34] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [30]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/05/30 23:18:30 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/30 23:18:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."
2024/05/30 23:18:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/mlops/anaconda3/envs/exp-tracking-env/lib/py

AssertionError: /home/mlops/anaconda3/envs/exp-tracking-env/lib/python3.9/distutils/core.py